# Мы изучили полносвязный слой Dence (В папке keras)
# Сейчас изучим СВЕРТОЧНЫЙ СЛОЙ (CNN - Conv2D)

Обычные нейронные сети плохо масштабируются для больших изображений.

Например, для изображения большего размера — 200х200х3 — количество весов станет равным 200х200х3 = 120000.
Более того, нам понадобится не один подобный нейрон, поэтому общее количество параметров модели (весов) начнет расти очень быстро.
Становится очевидным тот факт, что полносвязность избыточна, и **слишком большое количество параметров быстро приведет к переобучению.**

### Сверточный слой **Conv2D**
Существуют одномерные, двумерные и даже трехмерные сверточные слои. В данном уроке будут рассмотрены ***двумерные*** сверточные слои.

Для моделей нейронных сетей, построенных на основе класса Sequential, для создания слоя используйте конструкцию:

```.add(Conv2D(32, (3, 3), padding='same', activation='relu', strides=(1,1))```

* Первый параметр 32 - количество ядер (фильтров) свертки.
* Второй параметр (3, 3) – размер ядра свертки.
* Третий параметр padding='same' - тип заполнения краев, нужен для сохранения размеров изображения, по умолчанию значение padding='valid'.
* Четвертый параметр activation='relu' – указание функции активации.
* Пятый параметр strides=(1, 1) – необязательный, задает шаг смещения фильтра (подробнее в следующем разделе).


---

Дополнительная информация ([База знаний УИИ - **«Функции активации»**](https://colab.research.google.com/drive/1pGc7CFdrkKBhcXLqZNUzLXH4N83rRAl7?usp=sharing))

---

---
Дополнительная информация ([База знаний УИИ - **«Сверточный слой Conv2D»**](https://colab.research.google.com/drive/1bQNGBTEqen_QiYBDGqQ5Uu3iqet_G5ps?usp=sharing))

---


#### **Различия полносвязного и сверточного слоев**

Принципиальное различие между полносвязным и сверточным слоем состоит в том, что:
    
Полносвязные слои изучают глобальные характеристики изображения целиком
Серточные слои изучают локальные шаблоны в небольших фрагментах изображения

**Минус полносвязного слоя в том, что:**

- он обладает низкой вариативностью. То, что мы будем подавать на вход, должно быть практически идентично тому, на чем обучали, даже смещение объекта на листе уже критично.Локализованные признаки он не обнаружит.
- при использовании полносвязного слоя каждый нейрон связан с каждым нейроном предыдущего слоя, т.е. каждый нейрон анализирует все изображение и может найти ложные зависимости.

Вариант решения: создаем "маленький Dense" и им пробегаем по всему изображению в поисках совпадения. Это позволит обнаружить объект, даже если он уменьшен или смещен.

#### **Принцип работы сверточного слоя**

Основная задача сверточного слоя – выделить признаки во входном изображении ядрами свертки (фильтрами) и сформировать карту признаков в виде тензора. 

**Карта признаков или Карта активации** - это обработанное ядром свертки исходное изображение.
 
**Ядра свертки (фильтры)** - это тензоры одного размера (задается при настройке слоя). Количество ядер в слое определяет глубину выходного массива (т.е. количество ядер вместе с разрешением изображения определяют форму выходного тензора). 
 
**Свёртка** – это операция вычисления нового значения на основе значения выбранного пикселя и значений окружающих его пикселей.

---

Алгоритм свёртки можно описать так:
- фильтр накладывается на левую верхнюю часть входящего изображения;
- производится поэлементное умножение значений фильтра и значений пикселей изображения;
- полученные значения складываются, сумма будет результатом свертки области наложения (одно число);
- фильтр перемещается дальше по изображению (за смещение  отвечает параметр **strides**, по умолчанию равен **(1,1)**, т.е. смещение на 1 пиксель вправо, при достижении конца строки сдвиг вниз на 1 пиксель, и вновь с начала строки);
- в новом положении окна фильтра производится поэлементное умножение значений фильтра ...
- повтор до тех пор, пока аналогичным образом не будут обработаны все участки.


#### **Заполнение (Padding)**

Когда мы выбираем достаточно малый размер ядра фильтра, потеря в размерах выходного слоя невелика. Если же это значение увеличивается, то и размеры выходного слоя уменьшается по отношению к оригинальному размеру входного изображения. Это может стать проблемой, особенно если в модели нейронной сети подключено последовательно много сверточных слоев.

Чтобы избежать потерь в разрешении выходных изображений, в настройках сверточных слоев используют дополнительный параметр – заполнение (**padding**), позволяющий расширить полученное изображение по краям до того же размера, что и на входе свертки.

**Решение:** добавляем параметр **padding='same'**, тогда Keras дополнит выходное изображение по внешней границе нулями (другие числа исказили бы информацию, а нули дают только изменение размера), расположит их так, чтобы выходной массив не потерял в размерах. Заполнение распределяется по границам исходя из того, как много потерь нужно возместить.

#### **Размеры ядра свертки**

Свёрточные нейронные сети используют допущение, что входные данные — изображения, поэтому они образуют более чувствительную архитектуру к подобному типу данных. 

В частности, в отличие от обычных нейронных сетей, слои в свёрточной нейронной сети располагают ядра свертки в трех измерениях — ширине, высоте, глубине (здесь термин "глубина" относится к третьему измерению ядер, а не глубине самой нейронной сети, измеряемой в количестве слоёв).

Например, входные изображения из набора данных **CIFAR-10** являются входными данными в 3D-представлении, форма которых равна **32х32х3** (ширина, высота, глубина). Как мы увидим позднее, нейроны в одном слое будут связаны с небольшим количеством нейронов предыдущего слоя, вместо того чтобы быть связанными с ними всеми.

Таким образом, глубина фильтров первого слоя совпадает с количеством каналов входного изображения. Если на вход свёрточному слою подаётся RGB изображение (**3** канала) и требуется получить **32** карты признаков, то свёрточный слой должен содержать в себе **32** фильтра глубиной **3**.


Веса фильтра, который пробегает по изображению, не изменяются во время самого прохождения, но обучаются от батча к батчу. В результате обучения фильтр начинает отвечать за поиск определенного признака. в конце концов  одно ядро будет выделять горизонтальную прямую, другое – вертикальную, третье – диагональ и т.п. 


Для обработки изображений (не только нейросетями) нередко требуется решать конкретные задачи: выделять границы, повышать резкость или применять размытие. 

Для подобных целей были получены сверточные фильтры, которые сегодня часто встраиваются в различные графические редакторы. 

Самые распространенные размеры ядра двумерных сверточных слоев – **3х3**, **5х5** и **7х7**. Количество ядер, как правило, кратно двум – **8, 16, 32, 64** и т.д. Но никто не запрещает использовать ядро **3х7** или **27х27** и количество ядер **315**. 

Например, такие ядра свертки используются в фильтрах Photoshop.


### Слой подвыборки (**Pooling**-слой)
Основные задачи слоев типа **Pooling**:
1. Распознавание объектов вне зависимости от масштаба;
2. Факт наличия признака важнее знания места его точного положения на изображении.

Этот слой немного похож на сверточный, поскольку у него тоже есть ядро (окно фильтра). Но, в отличие от сверточного слоя, он уменьшает размер изображения, выбирая максимальное (**MaxPooling**), среднее (**AveragePooling**) или суммарное (**SumPooling**) значение из окна фильтра. В некотором смысле слой подвыборки делает информацию более сконцентрированной, обобщенной.


Слой подвыборки имеет один обязательный параметр — **pool_size** (размер окна подвыборки), и один необязательный - **strides** (шаг смещения окна). Причем если **strides** не указан, то по умолчанию **strides=pool_size**, то есть окно смещается на размер фильтра.

Наиболее часто используется слой **MaxPooling** с ядром **(2, 2)** и смещением по умолчанию - он уменьшает размеры входного тензора по ширине и высоте в два раза. Поэтому рекомендуется использовать такие разрешения изображений, чтобы размеры по обоим измерениям делились на **2** без остатка, иначе при уменьшении размеров часть информации будет потеряна.

> Если "сжать" слоем **MaxPooling** изображение размером **15 x 15**, на выходе получим **7 x 7**. 
А при "разжатии" изображения слоем **Upsampling** или **Conv2DTranspose** (выполняют что-то вроде обратной к **MaxPooling** операции) получим **14 x 14**, что не совпадет с исходными размерами.

Некоторые библиотеки позволяют задавать раздельные параметры уменьшения по высоте и ширине, создавать прямоугольное ядро подвыборки. Однако чаще всего оно квадратное.

Чтобы добавить слой, используйте: 

    .add(MaxPooling2D(pool_size=(2, 2)), где (2,2) – размер окна, в котором выбирается максимальное значение.

### Финальная классификация данных

После прохождения через сверточные слои и слои подвыборки данные необходимо систематизировать и классифицировать. Для этого применяют полносвязные слои **Dense**, которые вы изучили на предыдущих уроках.

Чтобы правильно передать данные от сверточного слоя на полносвязный, нужно сделать данные одномерными. Для данной задачи подходят слои:
- `Flatten()` - "сплющивает" многомерные входные данные в одномерный вектор, при этом размеры данных по всем осям перемножаются; дополнительных параметров нет. Например, входящее изображение формы **(28, 28, 3)** преобразуется в вектор формы **(2352)**.
- `Reshape(...)` - смена формы данных. Требуется указать в скобках желаемую форму данных. Позволяет не только вытягивать данные в вектор, но и произвольно менять форму, например **(28, 28, 3)** преобразовать в **(3, 784)**, или в **(14, 14, 12)**. Объем формы данных (произведение размеров по всем осям) на входе должен совпадать с объемом желаемой формы данных.

В зависимости от задачи, выходной **Dense**-слой может вычислять вероятности для каждого класса или выдавать номер (метку) класса.

## Создание простой модели сверточной нейронной сети

In [ ]:
# Подключите основу – класс создания последовательной модели **Sequential**:
from tensorflow.keras.models import Sequential

# С помощью него создайте экземпляр вашей модели:
model = Sequential()

Это и есть ваша модель! Сейчас она больше похожа на пустую коробку. Чтобы она что-то делала, нужно поместить в нее какой-нибудь механизм. Это не механизм в обычном смысле слова, потому что вы будете оперировать не предметами, а информацией – главным ресурсом XXI века. Механизм будет принимать на вход и выдавать на выход какие-то данные.

Так из чего же вы можете создать механизм? Для начала определитесь, сколько информации вы будете давать нейросети на вход. Один экземпляр такой информации называется **объектом**. Не углубляйтесь пока, какими они бывают и как устроены. Сейчас достаточно знать, что объекты всегда состоят из чисел.

Например, вы решили, что ваши объекты - изображения. Для подачи в нейросеть их надо оцифровать. 

У изображений есть высота **img_height**, ширина **img_width** и количество цветовых каналов **channels**.

Их называют входной формой (или формой входных данных) и записывают как:

    input_shape=(img_height, img_width, channels)

---

**Важно**: все изображения, которые вы подаете на вход нейронной сети, должны иметь общие высоту, ширину и количество каналов. Ниже вы узнаете, как это сделать. 

---

In [ ]:
# Добавьте в модель первый слой при помощи .add():

from tensorflow.keras.layers import Conv2D

# Первый сверточный слой
model.add(Conv2D(8, (3, 3), padding='same', activation='relu', input_shape=(28, 56, 3)))

Расшифруем написанное выше: первый сверточный слой принимает на вход цветное изображение (**3** канала) размерами **28** на **56** пикселей. То есть форма входящего массива - **(28, 56, 3)**.

Внутри слоя к нему применяется свертка **8**-ю фильтрами **(3, 3)** с шагом смещения **(1, 1)**, а затем функция активации **relu**.

Какой формы получится выходной массив? 

Обратите внимание, что **padding ='same'**, **stride=(1,1)** по умолчанию; вычислим `pad = (size - 1) / 2 = (3-1) /2 = 1`. 

Используем формулу:

    output_h = (input_h + 2 * pad - size) // stride + 1 = (28 + 2 * 1 - 3) // 1 + 1 = 27 + 1 = 28
    output_w = (input_w + 2 * pad - size) // stride + 1 = (56 + 2 * 1 - 3) // 1 + 1 = 55 + 1 = 56

Как видите, при стандартном **stride** и **padding ='same'** длина и ширина входного и выходного массивов сверточного слоя равны. Но отличие все-таки будет - это глубина!

На вход пришло **3** канала, а сверточный слой имеет **8** ядер свертки, каждое из которых выдает свою карту признаков, обработав любое количество каналов. 
Значит, глубина на выходе будет **8** вместо **3**. Полная форма данных на выходе получится **(28, 56, 8)**.

Проверьте это методом модели `.summary()`:


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   

 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
=================================================================
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________

In [ ]:
# Посмотрим, что будет, если добавить следующий сверточный слой:
# c 5 фильтрами с ядром (3, 2),
# шагом смещения (2, 3)
# padding ='valid':

# Второй сверточный слой
model.add(Conv2D(5, (3, 2), strides = (2,3), padding='valid', activation='relu'))

Если **padding='valid'**, то по правилам **pad = 0**. Подставим значения:
 
        output_h = (input_h + 2 * pad - size) // stride + 1 = (28 + 2 * 0 - 3) // 2 + 1 = 25 // 2 + 1 = 12 + 1 = 13
        output_w = (input_w + 2 * pad - size) // stride + 1 = (56 + 2 * 0 - 2) // 3 + 1 = 54 // 3 + 1 = 18 + 1 = 19
 
У вас **5** фильтров, значит форма данных на выходе получится **(13, 19, 5)**.

Проверьте:

```model.summary()```




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   

 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 19, 5)         245       
                                                                 
=================================================================
Total params: 469
Trainable params: 469
Non-trainable params: 0
_________________________________________________________________

In [ ]:
# Примените слой MaxPooling2D:

from tensorflow.keras.layers import MaxPooling2D 

# Слой подвыборки
model.add(MaxPooling2D(pool_size=(3, 3)))

**MaxPooling2D** изменит форму данных следующим образом (учитывая, что **stride=pool_size**): 

     output_h = (input - pool_size) // strides + 1 = (13 - 3) // 3 + 1 = 4 
     output_w = (input - pool_size) // strides + 1 = (19 - 3) // 3 + 1 = 6

Глубина в **MaxPooling2D** не меняется, выходная форма данных **(4, 6, 5)**.

Проверьте:

In [ ]:
model.summary()

In [ ]:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 19, 5)         245       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 6, 5)          0         
 )                                                               
                                                                 
=================================================================
Total params: 469
Trainable params: 469
Non-trainable params: 0
_________________________________________________________________

Далее примените слой Flatten, который вытягивает входящий тензор в одномерный вектор. На входе слоя ожидается тензор (4, 6, 5), а на выходе будет вектор (4 * 6 * 5) = (120)

In [ ]:
from tensorflow.keras.layers import Flatten

# Слой преобразования многомерных данных в одномерные 
model.add(Flatten())
model.summary()

In [ ]:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 19, 5)         245       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 6, 5)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 120)               0         
                                                                 
=================================================================
Total params: 469
Trainable params: 469
Non-trainable params: 0
_________________________________________________________________

Нужно обратить внимание, что размерность первых двух элементов тензора перед Flatten слоем (в этом случае, 4 х 6) не должна быть очень большая - можно добавлять слои Conv2D и MaxPooling2D пока эта размерность не станет равна 1 х 1. Например, если бы мы не добавили слой MaxPooling2D, эта размерность была бы равна 13 х 19, что бы было уже однозначно много.

Если размерность, которая мы подаем в Flatten() слой слишком большая, следующий слой не сможет извлечь достаточно значемые признаки для правильной классификации, потому что на вход слоя приходят слишком много данных.

________________________

Далее мы создадим последний, выходной слой **Dense**. 
Он получит на вход одномерный вектор **(120)**, а на выходе выдаст одномерный вектор **(3)**. Активационная функция **softmax** выдаст вероятности принадлежности входных данных к каждому из трех классов.

Проверьте:

In [ ]:
from tensorflow.keras.layers import Dense

model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 19, 5)         245       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 6, 5)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 120)               0         
                                                                 
 dense (Dense)               (None, 3)                 363       
                                                                 
=================================================================
Total params: 832
Trainable params: 832
Non-trainable params: 0
_________________________________________________________________